# 리뷰 크롤링 - 호텔스닷컴
- 호텔에 대한 리뷰를 크롤링하려고 하였다. 호텔스닷컴 사이트는 페이지별로 호텔에 대한 정보가 존재하는것이 아닌 사이트의 휠을 내릴경우 호텔이 추가 된다. 즉 css_selector로 호텔의 정보를 추출하려고 하였지만 6개의 호텔 정보만 존재한다. <br> 
그래서 selenium을 이용하여 자동으로 마우스 휠을 내려 호텔의 정보를 업데이트 하는 방식으로 진행하였다
- 다음으로 호텔의 정보가 url로 연결되어 있으며 리뷰들은 팝업의 형태로 나타내 이를 자동으로 크롤링하기 위해 다음과 같이 진행하였다.

In [1]:
import requests
import lxml.html
from selenium.webdriver import Chrome
import pandas as pd

In [132]:
browser = Chrome()

In [3]:
url = 'https://kr.hotels.com/search/search.html?resolved-location=CITY%3A759818%3AUNKNOWN%3AUNKNOWN&f-star-rating=5,4,3&destination-id=759818&q-destination=%EC%84%9C%EC%9A%B8,%20%ED%95%9C%EA%B5%AD&q-check-in=2020-05-13&q-check-out=2020-05-14&q-rooms=1&q-room-0-adults=2&q-room-0-children=0&sort-order=STAR_RATING_HIGHEST_FIRST'

In [64]:
requests.get(url)

<Response [200]>

In [133]:
browser.get(url)

In [134]:
import time  # 모든 호텔을 browser에 띄우기 위함
SCROLL_PAUSE_TIME = 5


last_height = browser.execute_script("return document.body.scrollHeight-1000")         

while True:
                                                     
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight-1000);")


    time.sleep(SCROLL_PAUSE_TIME)                                                
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight-1000);")  
    time.sleep(SCROLL_PAUSE_TIME)

           
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:                                               
        break

    last_height = new_height


In [151]:
hotel_reviews = []
hotels = browser.find_elements_by_css_selector('a.property-name-link')

In [152]:
#호텔 사이트
for i in range(len(hotels)) :
    time.sleep(1)
    hotels[i].click()
    time.sleep(2)
    browser.switch_to_window(browser.window_handles[1])   # 새로운 탭으로 이동 
    time.sleep(1)
    hotel_name = browser.find_element_by_tag_name('h1').text
    time.sleep(2)
    try :
        browser.find_elements_by_css_selector('a.total-reviews')[0].click() # 리뷰 이동
        time.sleep(1)
    except : 
        browser.close()
        browser.switch_to_window(browser.window_handles[0])
        continue
        
        
    while 1:
        try :
            reviews = browser.find_elements_by_css_selector('div.brand-reviews-listing>div.review-card>blockquote')
            ratings = browser.find_elements_by_css_selector('div.brand-reviews-listing>div.review-card> div.card-header>span.rating >span.rating-score')
            for j in range(len(reviews)) :
                review = reviews[j].text
                rating = ratings[j].text
                hotel_reviews.append([hotel_name,review,rating])
            browser.find_element_by_css_selector('div.pagination > div > a.cta.cta-secondary.cta-next').click()         # 다음페이지 
            time.sleep(3)
        except :
            browser.close()
            break
    browser.switch_to_window(browser.window_handles[0])



C:\Users\dhgus\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use driver.switch_to.window instead
  
C:\Users\dhgus\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: use driver.switch_to.window instead
C:\Users\dhgus\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: use driver.switch_to.window instead
  from ipykernel import kernelapp as app


In [153]:
result = pd.DataFrame(hotel_reviews, columns = ['호텔이름','리뷰', '평점'])

In [154]:
result.to_excel('crawl.xlsx', encoding = 'utf-8',index = False) 
# 일본어 중국어 영어가 섞여서 utf-8로 encoding 하였다 .